# 2021 Gender


In [40]:
import pandas as pd
import numpy as np
import math
import cleandata as cd

In [41]:
df = pd.read_csv('../data/Gemeinde_Exekutive - daten.csv')

## Clean Data

In [42]:
df = cd.clean(df)

Jahrgänge nicht zugeordnet: 15
Partei nicht zugeordnet: 9
Keine Jahrgänge: 21


In [54]:
df[df.Gemeinde == 'Stallikon']

,Gemeinde,Name,Name_cleaned,Jahrgang,Geschlecht,Partei,Funktion,Hinweis,jahrgang_nicht_zugeordnet,partei_nicht_zugeordnet,Gecheckt 2021,Alter,partei_c
43,Stallikon,"Werner Michel, 1949, parteilos",Werner Michel,1949.0,m,parteilos,chair,NaN,False,False,x,72.0,parteilos
44,Stallikon,"Robert Sidler, 1955, SVP",Robert Sidler,1955.0,m,SVP,NaN,NaN,False,False,x,66.0,svp
45,Stallikon,"Monika Rohr, 1952, FDP",Monika Rohr,1952.0,w,FDP,NaN,NaN,False,False,x,69.0,fdp
46,Stallikon,"Valerie Battiston, 1970, parteilos",Valerie Battiston,1970.0,w,parteilos,NaN,NaN,False,False,x,51.0,parteilos
47,Stallikon,"Remo Hablützel, 1983, FDP",Remo Hablützel,1983.0,m,FDP,NaN,NaN,False,False,x,38.0,fdp
48,Stallikon,"Ingrid Spiess, 1956, EVP",Ingrid Spiess,1956.0,w,EVP,NaN,NaN,False,False,x,65.0,evp


## Geschlechterverhältnis

In [43]:
grGender = df.groupby('Geschlecht').agg({'Name': 'count'})
grGender['prozent'] = round(100 / len(df) * grGender.Name, 1)
grGender

,Name,prozent
Geschlecht,,
m,737,72.2
w,282,27.6


In [36]:
len(df)

1021

## Frauen und Präsidenten - Export für DataWrapper

In [52]:
df_1 = df.groupby('Geschlecht').agg({'Name': 'count'})
df_1.rename(columns={'Name': 'In der Exekutiven'}, inplace=True)

df_p = df[df.Funktion == 'chair'].groupby('Geschlecht').agg({'Name': 'count'})
df_p.rename(columns={'Name': 'Im Präsidium'}, inplace=True)

df_1 = df_1.join(df_p).reset_index()

df_1['Geschlecht'] = df_1['Geschlecht'].apply(lambda x: 'Männer' if x == 'm' else 'Frauen')

df_1.to_csv('../export/gender.csv', index=False)

In [45]:
df.head()

,Gemeinde,Name,Name_cleaned,Jahrgang,Geschlecht,Partei,Funktion,Hinweis,jahrgang_nicht_zugeordnet,partei_nicht_zugeordnet,Gecheckt 2021,Alter,partei_c
0,Zell (ZH),Stefan Deinböck,Stefan Deinböck,1967.0,m,FDP,NaN,NaN,False,False,x,54.0,fdp
1,Zell (ZH),Regula Ehrismann,Regula Ehrismann,1964.0,w,EVP,chair,NaN,False,False,x,57.0,evp
2,Zell (ZH),Patricia Heuberger,Patricia Heuberger,1979.0,w,SP,NaN,NaN,False,False,x,42.0,sp
3,Zell (ZH),Markus Kernen,Markus Kernen,1962.0,m,BGV,NaN,NaN,False,False,x,59.0,verschiedene
4,Zell (ZH),Susanne Stahl,Susanne Stahl,1958.0,w,SP,NaN,NaN,False,False,x,63.0,sp


## Der Durchschnitts-Gemeinderat (m/w) ist...

In [33]:
df_alter = df[df.Alter.notna() & df.jahrgang_nicht_zugeordnet.notna()]
print("Männlich")
print("%s Jahre alt" % round(df_alter.Alter.mean(), 1))
print("parteilos")


Männlich
55.7 Jahre alt
parteilos


## Die Durchschnitts-Gemeinderätin (w) ist...

In [31]:
df_alter = df[df.Alter.notna() & df.jahrgang_nicht_zugeordnet.notna()]
print("%s Jahre alt" % round(df_alter[df_alter.Geschlecht == 'w'].Alter.mean(), 1))
print("parteilos")

54.7 Jahre alt
parteilos


## Der Durchschnitts-Gemeinderat (m) ist...

In [32]:
df_alter = df[df.Alter.notna() & df.jahrgang_nicht_zugeordnet.notna()]
print("%s Jahre alt" % round(df_alter[df_alter.Geschlecht == 'm'].Alter.mean(), 1))
print("parteilos")

56.0 Jahre alt
parteilos


## Gendereinseitige Gemeinderäte

In [13]:
df_g = pd.pivot_table(df, 'Name', 'Gemeinde', columns='Geschlecht', aggfunc='count')
df_g['w'] = df_g['w'].fillna(0)
df_g['total'] = df_g.m + df_g.w
df_g['m%'] = round(100 / df_g.total * df_g.m)
df_g['w%'] = round(100 / df_g.total * df_g.w)
print(len(df_g[df_g['m%'] == 100]))
df_g[df_g['m%'] == 100]

12


Geschlecht,m,w,total,m%,w%
Gemeinde,,,,,
Dachsen,5.0,0.0,5.0,100.0,0.0
Dietikon,7.0,0.0,7.0,100.0,0.0
Dägerlen,5.0,0.0,5.0,100.0,0.0
Ellikon an der Thur,5.0,0.0,5.0,100.0,0.0
Humlikon,5.0,0.0,5.0,100.0,0.0
Oetwil am See,7.0,0.0,7.0,100.0,0.0
Regensdorf,7.0,0.0,7.0,100.0,0.0
Wasterkingen,5.0,0.0,5.0,100.0,0.0
Weiach,5.0,0.0,5.0,100.0,0.0


### Die weiblichste Gemeinde

In [14]:
df_g = df_g.sort_values('w%', ascending=False)
df_g[df_g['w%'] > 50]

Geschlecht,m,w,total,m%,w%
Gemeinde,,,,,
Aeugst am Albis,3.0,5.0,8.0,38.0,62.0
Regensberg,2.0,3.0,5.0,40.0,60.0
Boppelsen,2.0,3.0,5.0,40.0,60.0
Kappel am Albis,2.0,3.0,5.0,40.0,60.0
Wiesendangen,3.0,4.0,7.0,43.0,57.0
Kilchberg (ZH),3.0,4.0,7.0,43.0,57.0
Marthalen,3.0,4.0,7.0,43.0,57.0
Uster,3.0,4.0,7.0,43.0,57.0
Rümlang,3.0,4.0,7.0,43.0,57.0


In [15]:
len(df_g[df_g['w%'] > 50])

10

## Gemeinderäte mit nur Männer, nur Frauen

In [34]:
df_g = pd.pivot_table(df, 'Name', 'Gemeinde', columns='Geschlecht', aggfunc='count')
df_g['w'] = df_g['w'].fillna(0)
df_g['total'] = df_g.m + df_g.w
df_g['m%'] = round(100 / df_g.total * df_g.m)
df_g['w%'] = round(100 / df_g.total * df_g.w)

print('Reine Männer-Räte: %s' % len(df_g[df_g['m%'] == 100]))
print('Reine Frauen-Räte: %s' % len(df_g[df_g['w%'] == 100]))

Reine Männer-Räte: 12
Reine Frauen-Räte: 0


## Gemeinderäte mit Mehrheit Frauen/Männer

In [13]:
df_g = pd.pivot_table(df, 'Name', 'Gemeinde', columns='Geschlecht', aggfunc='count')
df_g['w'] = df_g['w'].fillna(0)
df_g['total'] = df_g.m + df_g.w
df_g['m%'] = round(100 / df_g.total * df_g.m)
df_g['w%'] = round(100 / df_g.total * df_g.w)

print("Männer mehr 50 Prozent: %s" % len(df_g[df_g['m%'] > 50]))
print("Frauen mehr 50 Prozent: %s" % len(df_g[df_g['w%'] > 50]))
print("Männer und Frauen = 50 Prozent: %s" % len(df_g[df_g['w%'] == 50]))

Männer mehr 50 Prozent: 147
Frauen mehr 50 Prozent: 10
Männer und Frauen = 50 Prozent: 5


## Das Gemeindepräsidium ('chair') ist...

In [18]:
df_pr = pd.pivot_table(df, index='Funktion', columns='Geschlecht', values='Name', aggfunc='count')
#df_pr = df_pr[df_pr.Funktion == 'chair']
df_pr['m%'] = round(100 / (df_pr['m'] + df_pr['w']) * df_pr['m'])
df_pr['w%'] = round(100 / (df_pr['m'] + df_pr['w']) * df_pr['w'])

df_pr.head()

Geschlecht,m,w,m%,w%
Funktion,,,,
chair,133,29,82.0,18.0
school,3,6,33.0,67.0


## Jüngsten Frauen

In [ ]:
df[df.Geschlecht == 'w'].sort_values('Alter', ascending=True).head()

## Einzelne Frau im Gemeinderat?

In [51]:
df_w = pd.pivot_table(df, index='Gemeinde', columns='Geschlecht', values='Name', aggfunc='count')
len(df_w[df_w.w == 1])

61

## Gemeindepräsidentinnen vs Präsidenten

In [50]:
df[df.Funktion == 'chair'].groupby('Geschlecht').count()

,Gemeinde,Name,Name_cleaned,Jahrgang,Partei,Funktion,Hinweis,jahrgang_nicht_zugeordnet,partei_nicht_zugeordnet,Gecheckt 2021,Alter,partei_c
Geschlecht,,,,,,,,,,,,
m,133,133,133,131,132,133,5,133,133,133,131,133
w,29,29,29,28,29,29,1,29,29,29,28,29


## Alter in 10 Jahren

In [109]:
df_a = df[df.Jahrgang.notna()].copy()
df_a['agerange'] = df_a['Alter'].apply(lambda x: "%s-%s" % (math.floor(x / 10) * 10, math.floor(x / 10) * 10 + 10))
#df_a.sort_values('Alter', ascending=True).head()

#df_a = pd.pivot_table(df_a, columns='Geschlecht', index='agerange', values='Name', aggfunc='count')
#df_a = df_a.groupby(['agerange', 'Geschlecht']).agg({'Name': 'count'}).reset_index()
df_a = df_a.groupby(['Alter', 'Geschlecht']).agg({'Name': 'count'}).reset_index()
df_a['index'] = df_a['Geschlecht'].apply(lambda x: 1 if x == 'm' else 0)
#df_a['index'] = df_a['ager'].apply(lambda x: 1 if x == 'm' else -1)

df_a.rename(columns={'Name': 'Anzahl'}, inplace=True)

df_a = df_a.fillna(0)

def f(row):
    if row['Geschlecht'] == 'm':
        return round(100 / df_a[df_a.Geschlecht == 'm']['Anzahl'].sum() * row['Anzahl'],1)
    else:
        return round(100 / df_a[df_a.Geschlecht == 'w']['Anzahl'].sum() * row['Anzahl'],1)

df_a['%'] = df_a.apply(f, axis=1)

df_a[df_a.Geschlecht == 'w'].to_csv('../export/age_dots.csv', index=False)

In [118]:
df_a = df[df.Jahrgang.notna()].copy()
df_a['agerange'] = df_a['Alter'].apply(lambda x: "%s-%s" % (math.floor(x / 10) * 10, math.floor(x / 10) * 10 + 10))

#df_a = df_a.groupby(['agerange', 'Geschlecht']).agg({'Name': 'count'}).reset_index()
df_a = pd.pivot_table(df_a, index='agerange', columns='Geschlecht', values='Name', aggfunc='count')

df_a['m%'] = round(100 / df_a['m'].sum() * df_a['m'], 1)
df_a['w%'] = round(100 / df_a['w'].sum() * df_a['w'], 1)

df_a

Geschlecht,m,w,m%,w%
agerange,,,,
20-30,5,1,0.7,0.4
30-40,31,9,4.3,3.3
40-50,129,52,17.8,18.8
50-60,282,132,39.0,47.8
60-70,242,78,33.5,28.3
70-80,34,4,4.7,1.4


In [81]:
df_a = df[df.Jahrgang.notna()].copy()

#df_a = pd.pivot_table(df_a, columns='Geschlecht', index='Alter', values='Alter')

df_a = df_a[df_a.Geschlecht == 'w']

df_a = df_a[['Alter']]

df_a.head(20)
df_a.to_csv('../export/age_dots.csv', index=False)

## Wieviel Bevölkerung wird representiert?

In [19]:
# Load Bevölkerung
df_bev = pd.read_csv('../data/KANTON_ZUERICH_99.csv', sep=';', encoding='utf-8')
df_bev = df_bev[(df_bev.INDIKATOR_JAHR == 2020) & (df_bev.BFS_NR != 0)]

# Group Gemeinderäte
df_a = pd.pivot_table(df, columns='Geschlecht', index='Gemeinde', values='Name', aggfunc='count')

# Prozentualisieren
df_a['total'] = df_a['m'] + df_a['w']
df_a['w'] = df_a['w'].fillna(0)
#df_a['w%'] = round(100 / df_a['total'] * df_a['w'], 1)

# Join
df_a = df_a.reset_index()
df_bev['GEBIET_NAME'] = df_bev['GEBIET_NAME'].str.replace('a.A.', 'am Albis')
df_bev['GEBIET_NAME'] = df_bev['GEBIET_NAME'].str.replace(' ZH', ' (ZH)')
df_a = df_a.merge(df_bev, left_on = 'Gemeinde', right_on = 'GEBIET_NAME', how='left')

df_a['represent'] = df_a.apply(lambda row: row['INDIKATOR_VALUE'] / row['total'] * row['w'], axis=1)
df_a['represent'] = round(df_a['represent'])

w_represented = df_a['represent'].sum()
w_total = df_bev['INDIKATOR_VALUE'].sum()
w_not = round(100 / w_total * (w_total - w_represented))


print("Frauen repräsentieren: %s" % w_represented)
print("Frauen insgesamt: %s" % w_total)
print("So viele Frauen sind representiert: %s" % (100 - w_not))
print("So viele Frauen sind nicht representiert: %s" % w_not)

Frauen repräsentieren: 185766.0
Frauen insgesamt: 778364
So viele Frauen sind representiert: 24
So viele Frauen sind nicht representiert: 76


In [178]:
len(df_bev[])

162